In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import openweathermapy.core as owm
from datetime import date

from api_keys import api_key

from citipy import citipy

output_data_file = "output_data/cities.csv"


today = date.today()

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-180.000, high=180.000, size=6000)
lngs = np.random.uniform(low=-360.000, high=360.000, size=6000)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
settings = {"APPID": api_key, "units": "imperial", "lang": "EN"}
keys = ["clouds.all", "sys.country", "dt", "main.humidity", "coord.lat", "coord.lon","main.temp_max", "wind.speed"]
df = pd.DataFrame(columns=['City','Cloudiness', 'Country', 'Date', 'Humidity', 'Lat', 'Lng', 'Max Temp', 'Wind Speed'])

# Write a logfile of each city as it is being processed with the city name and number
logfile = open("output_data/cities.log", "w")

print("Beginning Data Retrieval")
print("-----------------------------")

i = 0
setCnt = 1

for city in cities:
    
    i = i + 1
    
    print(f"Processing Record {i} of Set {setCnt} | {city}")
    
    try:
        data = owm.get_current(city, **settings)
        df = df.append({'City':city, 'Cloudiness':data('clouds.all'), 'Country':data('sys.country'), 'Date':data('dt'),
                        'Humidity':data('main.humidity'),'Lat':data('coord.lat'),'Lng':data('coord.lon'),
                        'Max Temp': data("main.temp_max"),'Wind Speed': data("wind.speed")}, ignore_index=True)
        logfile.write(f"{city},{data('id')}\n")
    except:
        print("City not found. Skipping...")   

    if i == 25:
        print('Sleeping...')
        time.sleep(40)
        i = 0
        setCnt = setCnt + 1
    
print("-----------------------------")
print("Data Retrieval Complete ")
print("-----------------------------")

logfile.close()

In [ ]:
df.info()

In [ ]:

latitude_x = df['Lat']
max_temp_y = df['Max Temp']

# Generate the Graph
plt.scatter(
    latitude_x, 
    max_temp_y, 
    edgecolors="black", label="Test")

# Incorporate the other graph properties
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (F)')
plt.title(f'City Latitude vs. Max Temperature ({today.strftime("%m/%d/%y")})')
plt.grid()

# Save Figure
plt.savefig("./output_data/citymaxtemp.png")

plt